### This is pipeline for `RNA-Seq` analysis. For this, I have used `interactive jupyter-notebook`, where I can run any `shell script`, `R` code or `python` code. For know more about `interactive jupyter-notebook` you can go through following links. 

* https://jakevdp.github.io/PythonDataScienceHandbook/01.05-ipython-and-shell-commands.html
* https://stackoverflow.com/questions/53301306/access-variable-declared-in-a-bash-cell-from-another-jupyter-cell
* https://www.linkedin.com/pulse/interfacing-r-from-python-3-jupyter-notebook-jared-stufft/
* https://hbctraining.github.io/Intro-to-rnaseq-hpc-O2/lessons/02_assessing_quality.html
* https://bioinformatics.uconn.edu/resources-and-events/tutorials-2/rna-seq-tutorial-with-reference-genome/#
* https://towardsdatascience.com/guide-to-r-and-python-in-a-single-jupyter-notebook-ff12532eb3ba

In [43]:
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [2]:
%%R
library(edgeR)
library(RColorBrewer)
#library(pheatmap)
#library(EnhancedVolcano)
# library(goseq)
# library(ggplot2)
# library(dplyr)
# library(biomaRt)
# library(goseq)
#library(sratoolkit)

R[write to console]: Loading required package: limma



#### For this work, we have used 6 samples(SRR) from the study [SRP103945](https://www.ncbi.nlm.nih.gov/Traces/study/?acc=PRJNA382981&o=acc_s%3Aa). The 6 RNA-seq samples are `SRR5448863`, `SRR5448864`, `SRR5448865`, `SRR5448866`,`SRR5448867`, and `SRR5448868` . To know about more about the samples and study please go through the [link1](https://www.refine.bio/experiments/SRP103945/rna-seq-and-chip-seq-analysis-of-bmi1-or-ring1b-silenced-prostate-cancer-cells-c4-2) and [link2](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE97831).

* https://linsalrob.github.io/ComputationalGenomicsManual/Databases/SRA.html

### Assign variables for the entire pipeline. 

In [48]:
import rpy2.rinterface
import os
home_dir= os.path.expanduser('~')
SRA_List=[ "SRR5448863","SRR5448864", "SRR5448866", "SRR5448868"]
SRA_1="SRR5448863"
FASTQ_DIR = home_dir+ "/ncbi/fastq/"
FASTQC_DIR = home_dir+ "/ncbi/fastqc/"
BOWTIE_SAM_OUT_DIR  = home_dir+"/ncbi/bowtie_output"
BAM_DIR  = home_dir+"/ncbi/BAM_files/"
ANNOTATION_FILE = home_dir+"/ncbi/reference/gencode.v39.annotation.gtf"
HT_SEQ_OUT_DIR= home_dir+"/ncbi/HT_Seq_Count/"

In [45]:
HT_SEQ_OUT_DIR

'/home/pdutta/ncbi/HT_Seq_Count/'

### `prefetch`: Download data from `NCBI Sequence Read Archive` in `.sra` format using FASP or HTTPS protocols

In [33]:
for sra in SRA_List:
    print("\nSRA: ",sra)
    ! prefetch $sra


SRA:  SRR5448863

2022-04-07T22:29:11 prefetch.2.8.0: 1) 'SRR5448863' is found locally

SRA:  SRR5448864

2022-04-07T22:29:11 prefetch.2.8.0: 1) Downloading 'SRR5448864'...
2022-04-07T22:29:11 prefetch.2.8.0:  Downloading via http...
2022-04-07T22:30:52 prefetch.2.8.0: 1) 'SRR5448864' was downloaded successfully

SRA:  SRR5448866

2022-04-07T22:30:53 prefetch.2.8.0: 1) Downloading 'SRR5448866'...
2022-04-07T22:30:53 prefetch.2.8.0:  Downloading via http...
2022-04-07T22:32:55 prefetch.2.8.0: 1) 'SRR5448866' was downloaded successfully

SRA:  SRR5448868

2022-04-07T22:32:55 prefetch.2.8.0: 1) Downloading 'SRR5448868'...
2022-04-07T22:32:55 prefetch.2.8.0:  Downloading via http...
2022-04-07T22:34:42 prefetch.2.8.0: 1) 'SRR5448868' was downloaded successfully


#### `fastq-dump` is a tool for downloading sequencing reads from NCBI’s Sequence Read Archive (SRA). These sequence reads will be downloaded as `FASTQ` files.
* https://rnnh.github.io/bioinfo-notebook/docs/fastq-dump.html
* https://www.reneshbedre.com/blog/ncbi_sra_toolkit.html

In [34]:
! mkdir -p -- "$FASTQ_DIR"
for sra in SRA_List:
    print("\nSRA: ",sra)
    ! fastq-dump --skip-technical --readids --read-filter pass --dumpbase --outdir "$FASTQ_DIR" --split-files "$sra"


SRA:  SRR5448863
Read 33558497 spots for SRR5448863
Written 33558497 spots for SRR5448863

SRA:  SRR5448864
Read 29781556 spots for SRR5448864
Written 29781556 spots for SRR5448864

SRA:  SRR5448866
Read 27465767 spots for SRR5448866
Written 27465767 spots for SRR5448866

SRA:  SRR5448868
Read 29232809 spots for SRR5448868
Written 29232809 spots for SRR5448868


#### `FastQC` provides a simple way to do some quality control checks on raw sequence data coming from high throughput sequencing pipelines. 

In [35]:
! mkdir -p -- "$FASTQC_DIR"
! fastqc -t 30 --outdir "$FASTQC_DIR" $FASTQ_DIR*.fastq

Started analysis of SRR5448863_pass_1.fastq
Started analysis of SRR5448863_pass_2.fastq
Started analysis of SRR5448864_pass_1.fastq
Started analysis of SRR5448864_pass_2.fastq
Started analysis of SRR5448866_pass_1.fastq
Started analysis of SRR5448866_pass_2.fastq
Started analysis of SRR5448868_pass_1.fastq
Started analysis of SRR5448868_pass_2.fastq
Approx 5% complete for SRR5448864_pass_1.fastq
Approx 5% complete for SRR5448863_pass_1.fastq
Approx 5% complete for SRR5448866_pass_1.fastq
Approx 5% complete for SRR5448864_pass_2.fastq
Approx 5% complete for SRR5448863_pass_2.fastq
Approx 5% complete for SRR5448866_pass_2.fastq
Approx 5% complete for SRR5448868_pass_1.fastq
Approx 5% complete for SRR5448868_pass_2.fastq
Approx 10% complete for SRR5448866_pass_1.fastq
Approx 10% complete for SRR5448864_pass_1.fastq
Approx 10% complete for SRR5448866_pass_2.fastq
Approx 10% complete for SRR5448868_pass_1.fastq
Approx 10% complete for SRR5448864_pass_2.fastq
Approx 10% complete for SRR54488

## Alignment using `bowtie2`.
* https://rnnh.github.io/bioinfo-notebook/docs/bowtie2.html
## To install `bowtie2` you can use any of two approaches
 * For installing using conda, try ```conda install -c bioconda/label/broken bowtie2```
 * For manual installation, please follow the link https://www.metagenomics.wiki/tools/bowtie2/install 

### Run the `script` file named `bowtie2.sh`. The file generats `.SAM` file.  

In [77]:
! head "$BOWTIE_SAM_OUT_DIR/output.sam"

@HD	VN:1.0	SO:unsorted
@SQ	SN:chr1	LN:248956422
@SQ	SN:chr2	LN:242193529
@SQ	SN:chr3	LN:198295559
@SQ	SN:chr4	LN:190214555
@SQ	SN:chr5	LN:181538259
@SQ	SN:chr6	LN:170805979
@SQ	SN:chr7	LN:159345973
@SQ	SN:chr8	LN:145138636
@SQ	SN:chr9	LN:138394717


### Now we will use `samtools` to generate `.bam` file from the generated `.sam` file 
* For install samtools, use the following conda command <br>
  `conda install -c bioconda samtools==1.15`

* https://rnnh.github.io/bioinfo-notebook/docs/samtools.html
* http://quinlanlab.org/tutorials/samtools/samtools.html

In [38]:
! mkdir -p -- "$BAM_DIR"
for sra in SRA_List:
    print("Using SAMTOOLS for" , sra )
    !samtools view -@ n -Sb -o "$BAM_DIR$sra".bam "$BOWTIE_SAM_OUT_DIR/$sra"_SAM.sam
    !samtools sort -O bam -o "$BAM_DIR"sorted_"$sra".bam "$BAM_DIR$sra".bam
    !samtools index "$BAM_DIR"sorted_"$sra".bam
#! samtools view sample.bam | head

Using SAMTOOLS for SRR5448863
[bam_sort_core] merging from 20 files and 1 in-memory blocks...
Using SAMTOOLS for SRR5448864
[main_samview] fail to read the header from "/home/pdutta/ncbi/bowtie_output/SRR5448864_SAM.sam".
[E::hts_open_format] Failed to open file "/home/pdutta/ncbi/BAM_files/SRR5448864.bam" : No such file or directory
samtools sort: can't open "/home/pdutta/ncbi/BAM_files/SRR5448864.bam": No such file or directory
[E::hts_open_format] Failed to open file "/home/pdutta/ncbi/BAM_files/sorted_SRR5448864.bam" : No such file or directory
samtools index: failed to open "/home/pdutta/ncbi/BAM_files/sorted_SRR5448864.bam": No such file or directory
Using SAMTOOLS for SRR5448866
[bam_sort_core] merging from 16 files and 1 in-memory blocks...
Using SAMTOOLS for SRR5448868
[bam_sort_core] merging from 17 files and 1 in-memory blocks...


### We have used `HtSeq` to count how many reads map for each feature. 
https://rnnh.github.io/bioinfo-notebook/docs/htseq-count.html


In [39]:
! mkdir -p -- "$HT_SEQ_OUT_DIR"
for sra in SRA_List:
    ! htseq-count --format bam "$BAM_DIR"sorted_"$sra".bam "$ANNOTATION_FILE" > "$HT_SEQ_OUT_DIR$sra"Htseq_count.txt

100000 GFF lines processed.
200000 GFF lines processed.
300000 GFF lines processed.
400000 GFF lines processed.
500000 GFF lines processed.
600000 GFF lines processed.
700000 GFF lines processed.
800000 GFF lines processed.
900000 GFF lines processed.
1000000 GFF lines processed.
1100000 GFF lines processed.
1200000 GFF lines processed.
1300000 GFF lines processed.
1400000 GFF lines processed.
1500000 GFF lines processed.
1600000 GFF lines processed.
1700000 GFF lines processed.
1800000 GFF lines processed.
1900000 GFF lines processed.
2000000 GFF lines processed.
2100000 GFF lines processed.
2200000 GFF lines processed.
2300000 GFF lines processed.
2400000 GFF lines processed.
2500000 GFF lines processed.
2600000 GFF lines processed.
2700000 GFF lines processed.
2800000 GFF lines processed.
2900000 GFF lines processed.
3000000 GFF lines processed.
3100000 GFF lines processed.
3200000 GFF lines processed.
3241002 GFF lines processed.
100000 alignment record pairs processed.
200000 alig

### `DESeq2` using `R`

For Python, you can follow this [link](https://github.com/wckdouglas/diffexpr)

In [140]:
%%R
library("DESeq2")

In [64]:
%%R -i home_dir
# Set the working directory
directory <- paste(home_dir,"/ncbi/count_temp", sep="")
print(directory)
setwd(directory)

[1] "/home/pdutta/ncbi/count_temp"


In [66]:
%%R
outputPrefix <- "Gmax_DESeq2"

sampleFiles<- c("SRR5448863Htseq_count.txt",
                "SRR5448866Htseq_count.txt")
sampleNames <- c("Leaf tissue 1","Leaf tissue 2")
sampleCondition <- c("ambient","elevated")
sampleTable <- data.frame(sampleName = sampleNames, fileName = sampleFiles, condition = sampleCondition)

treatments = c("ambient","elevated")
print(sampleTable)

     sampleName                  fileName condition
1 Leaf tissue 1 SRR5448863Htseq_count.txt   ambient
2 Leaf tissue 2 SRR5448866Htseq_count.txt  elevated


In [67]:
%%R
library("DESeq2")
library("S4Vectors")
ddsHTSeq <- DESeqDataSetFromHTSeqCount(sampleTable = sampleTable,
                                       directory = directory,
                                       design = ~ condition)
colData(ddsHTSeq)$condition <- factor(colData(ddsHTSeq)$condition,
                                      levels = treatments)

In [86]:
%%R
ddsHTSeq

class: DESeqDataSet 
dim: 61533 2 
metadata(1): version
assays(1): counts
rownames(61533): ENSG00000000003.15 ENSG00000000005.6 ...
  ENSG00000289643.1 ENSG00000289644.1
rowData names(0):
colnames(2): SRR5448863Htseq_count.txt SRR5448866Htseq_count.txt
colData names(1): condition


In [70]:
%%R
dds <- DESeq(ddsHTSeq)
#res <- results(dds)
#print(dds)

R[write to console]: estimating size factors

R[write to console]: estimating dispersions

R[write to console]: Error in checkForExperimentalReplicates(object, modelMatrix) : 

  The design matrix has the same number of samples and coefficients to fit,
  so estimation of dispersion is not possible. Treating samples
  as replicates was deprecated in v1.20 and no longer supported since v1.22.





Error in checkForExperimentalReplicates(object, modelMatrix) : 

  The design matrix has the same number of samples and coefficients to fit,
  so estimation of dispersion is not possible. Treating samples
  as replicates was deprecated in v1.20 and no longer supported since v1.22.


RInterpreterError: Failed to parse and evaluate line 'dds <- DESeq(ddsHTSeq)\n#res <- results(dds)\n#print(dds)\n'.
R error message: 'Error in checkForExperimentalReplicates(object, modelMatrix) : \n\n  The design matrix has the same number of samples and coefficients to fit,\n  so estimation of dispersion is not possible. Treating samples\n  as replicates was deprecated in v1.20 and no longer supported since v1.22.'

In [74]:
%%R -i home_dir
# Set the working directory
directory <- paste(home_dir,"/ncbi/count_temp", sep="")

In [83]:
%%R
sampleFiles <- grep("txt",list.files(directory),value=TRUE)
sampleCondition <- sub("(.*txt).*","\\1",sampleFiles)
sampleTable <- data.frame(sampleName = sampleFiles,
                          fileName = sampleFiles,
                          condition = sampleCondition)
sampleTable$condition <- factor(sampleTable$condition)

In [84]:
%%R
sampleTable

                 sampleName                  fileName                 condition
1 SRR5448863Htseq_count.txt SRR5448863Htseq_count.txt SRR5448863Htseq_count.txt
2 SRR5448866Htseq_count.txt SRR5448866Htseq_count.txt SRR5448866Htseq_count.txt


In [85]:
%%R
library("DESeq2")
ddsHTSeq <- DESeqDataSetFromHTSeqCount(sampleTable = sampleTable,
                                       directory = directory,
                                       design= ~ condition)
ddsHTSeq

class: DESeqDataSet 
dim: 61533 2 
metadata(1): version
assays(1): counts
rownames(61533): ENSG00000000003.15 ENSG00000000005.6 ...
  ENSG00000289643.1 ENSG00000289644.1
rowData names(0):
colnames(2): SRR5448863Htseq_count.txt SRR5448866Htseq_count.txt
colData names(1): condition
